In [ ]:
%pip install -r requirements.txt

In [ ]:
import requests as req
import pandas as pd
import os
import re
import glob
import subprocess
import zipfile
import  yfinance as yf
from typing import List, Dict
import beaapi as bea
from dotenv import load_dotenv
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
load_dotenv()
current_date = datetime.now().strftime('%m-%d-%Y')

In [ ]:
def fetch_ssga_excel(etf: str, subfolder: str) -> str:
    """
    Fetch the SSGA Excel file for a single ETF symbol.
    
    Parameters:
    etf (str): ETF symbol in lowercase (e.g., "xli", "xlk").
    subfolder (str): Directory in which to save the downloaded file.
    
    Returns:
    str: Path to the downloaded Excel file.
    """
    url = f'https://www.ssga.com/library-content/products/fund-data/etfs/us/holdings-daily-us-en-{etf}.xlsx'
    response = req.get(url)
    
    if response.status_code != 200:
        raise ValueError(f"Failed to download {etf}. HTTP status code: {response.status_code}")
    
    current_date = datetime.now().strftime('%m-%d-%Y')
    file_path = os.path.join(subfolder, f'{etf}-{current_date}.xlsx')
    
    with open(file_path, 'wb') as file:
        file.write(response.content)
    
    return file_path

def process_ssga_excel(file_path: str) -> pd.DataFrame:
    """
    Reads the ETF Excel file from SSGA, trims disclaimers, removes empty lines, and returns a DataFrame.

    Parameters:
    file_path (str): Local path to the downloaded Excel file.

    Returns:
    pd.DataFrame: Processed SSGA holdings data with columns [Name, Ticker, Identifier, SEDOL, Weight, Sector, ...].
    """
    # The file has disclaimers in the last rows, skiprows=4 means we skip the top disclaimers
    # usecols="A:H" to read only the first 8 columns
    df = pd.read_excel(file_path, skiprows=4, header=0, usecols="A:H")
    
    # If the disclaimers line is found, slice the DataFrame up to that row
    disclaimers_str = (
        "Past performance is not a reliable indicator of future performance. "
        "Investment return and principal value will fluctuate, so you may have a gain or loss when shares are sold. "
        "Current performance may be higher or lower than that quoted. All results are historical and assume the "
        "reinvestment of dividends and capital gains. Visit www.ssga.com for most recent month-end performance. "
    )
    drop_index = df[df['Name'] == disclaimers_str].index
    if not drop_index.empty:
        df = df[:drop_index[0]]
    
    # Remove last row if it's entirely NaN
    if not df.empty and df.iloc[-1].isna().all():
        df = df[:-1]
    
    return df

def fetch_and_process_etf_data(etf_list):
    """
    Download and parse SSGA Excel holdings for a list of ETF tickers.
    
    Parameters:
    etf_list (List[str]): e.g. ['XLI', 'XLK', 'XLE', 'XLB'] (uppercase or mixed case).
    
    Returns:
    List[pd.DataFrame]: A list of DataFrames, one for each ETF, in the same order as etf_list.
    """
    subfolder = 'SSGA Data'
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)

    ssga_df_list = []
    
    for etf_symbol in etf_list:
        etf_lower = etf_symbol.lower()
        # 1) Download the Excel
        file_path = fetch_ssga_excel(etf_lower, subfolder)
        # 2) Process / clean up the DataFrame
        df = process_ssga_excel(file_path)
        df['ETF'] = etf_symbol.upper()
        ssga_df_list.append(df)

    return ssga_df_list

etf_list = ['XLI', 'XLK', 'XLE', 'XLB']
ssga_df_list = fetch_and_process_etf_data(etf_list)

# Print tails to verify we have data for each
for idx, df in enumerate(ssga_df_list, start=1):
    print(f"\n=== SSGA DataFrame #{idx} (ETF: {etf_list[idx-1]}) ===")
    print(df.tail())


In [ ]:
def fetch_bea_json(api_key: str, years: str) -> dict:
    """
    Fetches GDP by Industry data from the BEA API in JSON format.

    Parameters:
    -----------
    api_key : str
        our BEA API key.
    years : str
        A comma-separated string of years (e.g., "2020,2021,2022,2023,2024").

    Returns:
    --------
    dict
        Parsed JSON response from the BEA API, or None if an error occurred.
    """
    base_url = "https://apps.bea.gov/api/data/"
    params = {
        "UserID": api_key,
        "method": "GetData",
        "datasetname": "GDPbyIndustry",
        "Frequency": "Q",
        "Year": years,
        "Industry": "ALL",
        "TableID": "ALL",
        "ResultFormat": "JSON"
    }
    try:
        response = req.get(base_url, params=params)
        response.raise_for_status()  # raises HTTPError if status != 200
        data = response.json()
        
        # Check for error key in the response
        if 'Error' in data.get('BEAAPI', {}):
            err = data['BEAAPI']['Error']
            print(f"API request failed with code {err['APIErrorCode']}: {err['APIErrorDescription']}")
            return None
        
        return data
    except req.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return None

def process_bea_json(raw_data: dict) -> pd.DataFrame:
    """
    Parses the raw JSON from BEA into a pivoted DataFrame, excluding certain irrelevant rows.

    Parameters:
    -----------
    raw_data : dict
        The JSON dictionary returned by fetch_bea_json.

    Returns:
    --------
    pd.DataFrame
        A pivoted DataFrame with columns for YearQuarter, and rows for each Industry/IndustryDescription.
        Certain 'unwanted' industry descriptions are filtered out.
        Returns None if the data is missing or invalid.
    """
    if not raw_data or 'BEAAPI' not in raw_data or 'Results' not in raw_data['BEAAPI']:
        print("Error: 'Results' key not found in the API response.")
        return None

    results = raw_data['BEAAPI']['Results']
    df_results = pd.DataFrame(results)
    if df_results.empty or 'Data' not in df_results.columns:
        print("Error: No 'Data' found in the Results.")
        return None

    # 'Data' itself is in a list of dictionaries
    nested_data = df_results.loc[0, "Data"]
    bea_df = pd.DataFrame(nested_data)
    
    # Fix possible typos in the column name
    if "IndustrYDescription" in bea_df.columns:
        bea_df.rename(columns={"IndustrYDescription": "IndustryDescription"}, inplace=True)

    # Convert DataValue to float
    bea_df['DataValue'] = bea_df['DataValue'].astype(float, errors='raise')

    # Create a YearQuarter column to pivot on
    bea_df['YearQuarter'] = bea_df['Year'].astype(str) + 'Q' + bea_df['Quarter']

    # Pivot: index by (Industry, IndustryDescription), columns=YearQuarter, values=DataValue
    pivot_df = bea_df.pivot_table(
        index=['Industry', 'IndustryDescription'],
        columns='YearQuarter',
        values='DataValue',
        aggfunc='sum'
    ).reset_index()

    # Filter out unwanted industry descriptions
    excluded_industries = [
        "Taxes on production and imports less subsidies",
        "Energy inputs",
        "Intermediate inputs",
        "Materials inputs",
        "Purchased-services inputs",
        "Value added",
        "Compensation of employees",
        "Gross operating surplus"
    ]
    filtered_df = pivot_df[~pivot_df['IndustryDescription'].isin(excluded_industries)]

    return filtered_df

def save_bea_raw_data(df: pd.DataFrame, folder: str = 'BEA Data') -> str:
    """
    Saves the DataFrame to an Excel file with a timestamp-based filename.

    Parameters:
    -----------
    df : pd.DataFrame
        The pivoted and filtered DataFrame from process_bea_json.
    folder : str
        Destination folder name.

    Returns:
    --------
    str : Path to the saved Excel file.
    """
    os.makedirs(folder, exist_ok=True)
    current_date = datetime.now().strftime('%m-%d-%Y')
    file_path = os.path.join(folder, f"bea-gdp-by-industry-raw-{current_date}.xlsx")
    df.to_excel(file_path, index=False)
    return file_path

def map_sectors(df: pd.DataFrame) -> pd.DataFrame:
    """
    Maps each IndustryDescription to a top-level sector (Technology, Materials, Energy, Industrials),
    returning only rows in those focus sectors.

    Parameters:
    -----------
    df : pd.DataFrame
        Pivoted DataFrame with columns [Industry, IndustryDescription, ...quarter columns... ].

    Returns:
    --------
    pd.DataFrame
        A copy of df with a new 'Sector' column. Rows outside the focus sectors are excluded.
    """
    sector_map = {
        "Technology": [
            "Computer and electronic products",
            "Computer systems design and related services",
            "Data processing, internet publishing, and other information services",
            "Information-communications-technology-producing industries"
        ],
        "Materials": [
            "Agriculture, forestry, fishing, and hunting",
            "Farms",
            "Forestry, fishing, and related activities",
            "Mining",
            "Mining, except oil and gas",
            "Support activities for mining",
            "Wood products",
            "Paper products",
            "Chemical products",
            "Plastics and rubber products",
            "Nonmetallic mineral products",
            "Primary metals",
            "Fabricated metal products"
        ],
        "Energy": [
            "Oil and gas extraction",
            "Petroleum and coal products",
            "Pipeline transportation"
        ],
        "Industrials": [
            "Construction",
            "Machinery",
            "Electrical equipment, appliances, and components",
            "Other transportation equipment",
            "Miscellaneous manufacturing",
            "Durable goods",
            "Wholesale trade",
            "Rail transportation",
            "Water transportation",
            "Truck transportation",
            "Transit and ground passenger transportation",
            "Other transportation and support activities",
            "Transportation and warehousing",
            "Warehousing and storage",
            "Waste management and remediation services"
        ]
    }

    # Helper to assign a top-level sector
    def get_sector(description: str) -> str:
        for sector, cat_list in sector_map.items():
            if description in cat_list:
                return sector
        return "Other"

    df_copy = df.copy()
    df_copy["Sector"] = df_copy["IndustryDescription"].apply(get_sector)

    focus_sectors = ["Technology", "Materials", "Energy", "Industrials"]
    df_filtered = df_copy[df_copy["Sector"].isin(focus_sectors)]
    return df_filtered

def fetch_bea_gdp_by_industry(api_key: str, years: str) -> pd.DataFrame:
    """
    High-level orchestrator that fetches BEA data, processes it, saves a raw Excel,
    maps top-level sectors, and saves a final filtered Excel.

    Parameters:
    -----------
    api_key : str
        our BEA API key.
    years : str
        Comma-separated years (e.g. "2020,2021,2022,2023,2024").

    Returns:
    --------
    pd.DataFrame
        The final DataFrame with top-level sectors included.
    """
    raw_json = fetch_bea_json(api_key, years)
    if raw_json is None:
        return None
    
    pivoted_df = process_bea_json(raw_json)
    if pivoted_df is None:
        print("Error: Could not process BEA data into pivoted DataFrame.")
        return None

    # Save the pivoted "raw" data
    _ = save_bea_raw_data(pivoted_df)  # optional to keep the file path

    # Map each IndustryDescription to a top-level sector
    final_df = map_sectors(pivoted_df)

    # Save a separate 'filtered' file
    subfolder = 'BEA Data'
    current_date = datetime.now().strftime('%m-%d-%Y')
    filtered_path = os.path.join(subfolder, f"bea-gdp-by-industry-filtered-{current_date}.xlsx")
    final_df.to_excel(filtered_path, index=False)

    return final_df


if __name__ == "__main__":
    
    api_key = os.environ.get("beakey")  # our key as a string
    years = "2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024"
    
    bea_df = fetch_bea_gdp_by_industry(api_key, years)
    if bea_df is not None:
        print(bea_df.tail())
    else:
        print("BEA GDP data retrieval or processing failed.")


In [ ]:
def setup_kaggle_data():
    # Step 1: Ensure the "Kaggle Data" directory exists
    kaggle_data_dir = "Kaggle Data"
    os.makedirs(kaggle_data_dir, exist_ok=True)
    
    # Step 2: Check if the .csv file already exists
    csv_exists = any(filename.endswith('.csv') for filename in os.listdir(kaggle_data_dir))
    
    if not csv_exists:
        # Step 3: Download the dataset using the Kaggle CLI
        dataset = "jakewright/9000-tickers-of-stock-market-data-full-history"
        subprocess.run(["kaggle", "datasets", "download", "-d", dataset])
        
        # Step 4: Unzip the downloaded file
        zip_filename = dataset.split('/')[-1] + ".zip"
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(kaggle_data_dir)
        
        # Step 5: Remove any files in the "Kaggle Data" directory that are not .csv files
        for filename in os.listdir(kaggle_data_dir):
            if not filename.endswith('.csv'):
                os.remove(os.path.join(kaggle_data_dir, filename))
        
        # Remove the downloaded zip file
        os.remove(zip_filename)

# Call the function
setup_kaggle_data()

# Load the CSV file into a DataFrame
kag_df = pd.read_csv('Kaggle Data/all_stock_data.csv')

# Convert the 'Date' column to datetime format
kag_df['Date'] = pd.to_datetime(kag_df['Date'])

# Display the first few rows of the DataFrame
kag_df.head()

In [ ]:
# Filter the DataFrame for dates between 2010-01-01 and 2024-12-31
start_date = '2010-01-01'
end_date = '2024-12-31'
filtered_kag_df = kag_df[(kag_df['Date'] >= start_date) & (kag_df['Date'] <= end_date)]

# Display the first few rows of the filtered DataFrame
filtered_kag_df.head()

In [ ]:
def ensure_directory_exists(directory: str) -> None:
    """
    Ensure the specified directory exists, creating it if necessary.

    Parameters
    ----------
    directory : str
        The path of the directory to check or create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists.")

def download_data(tickers: List[str], start_date: str, end_date: str, interval: str = "1d") -> pd.DataFrame:
    """
    Download historical stock data for multiple tickers using yfinance.

    Parameters
    ----------
    tickers : List[str]
        A list of ticker symbols (e.g. ["XLI","XLK","XLE"]).
    start_date : str
        Start date in "YYYY-MM-DD" format.
    end_date : str
        End date in "YYYY-MM-DD" format.
    interval : str
        Data interval (e.g., "1d","1wk","1mo","3mo"). Defaults to "1d".

    Returns
    -------
    pd.DataFrame
        A multi-index DataFrame with columns as (Ticker, Field).
        By default, yfinance includes columns like "Open","High","Low","Close","Volume".
    """
    return yf.download(
        tickers=tickers,
        start=start_date,
        end=end_date,
        interval=interval,
        group_by="ticker",
        auto_adjust=True,
        threads=True,
        progress=True
    )

def filter_columns(df: pd.DataFrame, tickers: List[str], fields: List[str]) -> pd.DataFrame:
    """
    Select specific columns (fields) for each ticker from a multi-index DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        The multi-index DataFrame returned by download_data.
    tickers : List[str]
        The list of tickers to keep columns for.
    fields : List[str]
        The list of fields to keep for each ticker (e.g. ["High","Low"]).

    Returns
    -------
    pd.DataFrame
        A subset of the original DataFrame containing only the specified (Ticker, Field) columns.
    """
    # Build a list of valid (Ticker, Field) column tuples
    keep_cols = [(t, f) for t in tickers for f in fields if (t, f) in df.columns]
    return df[keep_cols].copy()

def add_change_columns(df: pd.DataFrame, tickers: List[str]) -> pd.DataFrame:
    """
    Add a "Change" column for each ticker, computed as (High - Low).

    Parameters
    ----------
    df : pd.DataFrame
        A multi-index DataFrame with columns like (Ticker, "High"), (Ticker, "Low").
    tickers : List[str]
        List of tickers to process.

    Returns
    -------
    pd.DataFrame
        Modified DataFrame with additional (Ticker, "Change") columns for each ticker having "High" and "Low".
    """
    for t in tickers:
        if (t, "High") in df.columns and (t, "Low") in df.columns:
            df[(t, "Change")] = df[(t, "High")] - df[(t, "Low")]
    return df

def process_change_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Restructure 'Change' columns from a multi-index to a single-level index.

    This extracts all (Ticker, "Change") columns, flattens them to 'Ticker_Change' columns,
    and resets the index so there is a single 'Date' column instead of the multi-index.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with columns like (Ticker, "Change").

    Returns
    -------
    pd.DataFrame
        A new DataFrame with each ticker's change in a single column named 'Ticker_Change',
        plus a 'Date' column (extracted from the index).
    """
    # Filter just the "Change" columns
    df_only_change = df.loc[:, (slice(None), "Change")].copy()  # multi-index slice
    # Remove the second level from column names
    df_only_change.columns = df_only_change.columns.droplevel(1)
    # Rename columns => Ticker_Change
    df_only_change.columns = [f"{ticker}_Change" for ticker in df_only_change.columns]
    df_only_change.reset_index(inplace=True)
    
    # Index column is named "index", renaming it to "Date"
    if df_only_change.columns[0] == "index":
        df_only_change.rename(columns={"index": "Date"}, inplace=True)
    return df_only_change

def save_to_csv(df: pd.DataFrame, path: str) -> None:
    """
    Save a DataFrame to CSV with no index, and print confirmation.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame to save.
    path : str
        Output CSV file path.
    """
    df.to_csv(path, index=False)
    print(f"Data with only 'Change' columns saved to: {path}")


if __name__ == "__main__":
    data_dir = "Yahoo Data"
    ensure_directory_exists(data_dir)

    tickers = ["XLI", "XLK", "XLE", "XLB"]
    # 1) Download data from yfinance
    df_full = download_data(tickers, start_date="2010-01-01", end_date="2024-01-01", interval="3mo")
    
    # 2) Filter columns to just 'High' and 'Low'
    df_filtered = filter_columns(df_full, tickers, ["High", "Low"])
    
    # 3) Add (Ticker,"Change") columns
    df_filtered = add_change_columns(df_filtered, tickers)
    
    # 4) Extract only the "Change" columns into a single-level DataFrame
    df_only_change = process_change_columns(df_filtered)
    
    # 5) Save the final result
    csv_path = os.path.join(data_dir, "sector_quarterly_only_change.csv")
    save_to_csv(df_only_change, csv_path)

    print(df_only_change.head())


In [ ]:
def extract_top_tickers_from_ssga(
    folder_path: str,
    n: int = 10,
    skiprows: int = 4,
    weight_col: str = "Weight",
    ticker_col: str = "Ticker"
) -> list:
    """
    Read all SSGA Excel files in a folder, extract the top n tickers by weight, and return a unique list of them.

    Parameters
    ----------
    folder_path : str
        Path to the folder containing .xlsx files from SSGA.
    n : int
        Number of top holdings to extract per file. Default is 10.
    skiprows : int
        Rows to skip at the top when reading the Excel. Default 4 based on our file format.
    weight_col : str
        Column name for weight in the Excel file. Default is "Weight".
    ticker_col : str
        Column name for ticker in the Excel file. Default is "Ticker".

    Returns
    -------
    List[str]
        A unique list of top ticker symbols across all SSGA Excel files in the folder.
    """
    ssga_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
    top_tickers = []

    for file in ssga_files:
        df = pd.read_excel(file, skiprows=skiprows)
        # Grab top n rows by weight, extract ticker col
        top_n = df.nlargest(n, weight_col)[ticker_col].tolist()
        top_tickers.extend(top_n)

    # Deduplicate
    return list(set(top_tickers))

def filter_kaggle_dataframe_by_tickers(
    df: pd.DataFrame,
    tickers: list,
    date_col: str = "Date",
    start_date: str = "2020-01-01",
    end_date: str = "2023-12-31"
) -> pd.DataFrame:
    """
    Filter a Kaggle-based DataFrame for the given tickers and a date range, returning a new DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame containing stock records (with a 'Ticker' column).
    tickers : list
        List of tickers to keep.
    date_col : str
        Name of the column holding dates. Default "Date".
    start_date : str
        Earliest date in "YYYY-MM-DD" format. Default "2020-01-01".
    end_date : str
        Latest date in "YYYY-MM-DD" format. Default "2023-12-31".

    Returns
    -------
    pd.DataFrame
        Filtered DataFrame with rows only for the specified tickers and date range.
        'Date' column is converted to datetime.
    """
    # Keep only rows that match top tickers
    df_filtered = df[df['Ticker'].isin(tickers)].copy()

    # Ensure date_col is datetime, then filter by date range
    df_filtered[date_col] = pd.to_datetime(df_filtered[date_col], errors='coerce')
    mask = (df_filtered[date_col] >= start_date) & (df_filtered[date_col] <= end_date)
    df_filtered = df_filtered[mask]

    return df_filtered

def resample_to_quarterly(
    df: pd.DataFrame,
    date_col: str = "Date",
    agg_dict: dict = None
) -> pd.DataFrame:
    """
    Resample a DataFrame (with Ticker & a date index) to quarterly frequency using the specified aggregations.

    Parameters
    ----------
    df : pd.DataFrame
        Stock DataFrame with columns including 'Ticker', 'Open','High','Low','Close','Volume','Dividends','Stock Splits'.
    date_col : str
        The column that will become the index for resampling. Default "Date".
    agg_dict : dict
        A dict mapping column -> aggregation function(s). If None is provided,
        a default is used for typical OHLC resampling.

    Returns
    -------
    pd.DataFrame
        Resampled DataFrame, grouped by Ticker, then aggregated quarterly, then reset to a normal index.
    """
    if agg_dict is None:
        agg_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum',
            'Dividends': 'sum',
            'Stock Splits': 'sum'
        }

    # Set the date_col as index, group by Ticker, then resample
    df_copy = df.copy()
    df_copy.set_index(date_col, inplace=True)
    # groupby Ticker, resample quarterly
    quarterly_df = (df_copy.groupby('Ticker')
                            .resample('Q')
                            .agg(agg_dict)
                            .reset_index())
    return quarterly_df


if __name__ == "__main__":
    # 1) Extract top 10 tickers from SSGA Excel files
    folder_path = "/home/pinagm/dev/Udacity_DataAnalyst_NanoDegree/D502 - Capstone Project/SSGA Data"
    top_tickers = extract_top_tickers_from_ssga(folder_path, n=10)
    print(f"Top tickers from SSGA: {top_tickers}")

    # 2) Filter an existing Kaggle-based DataFrame with top_tickers & date range
    filtered_kag_df = pd.read_csv("Kaggle Data/all_stock_data.csv")

    df_top = filter_kaggle_dataframe_by_tickers(
        filtered_kag_df,
        top_tickers,
        date_col="Date",
        start_date="2010-01-01",
        end_date="2024-12-31"
    )

    # 3) Resample that filtered DataFrame to quarterly frequency
    quarterly_kag_df = resample_to_quarterly(df_top)

    # 4) Preview & save the quarterly DataFrame
    print(quarterly_kag_df.head())
    output_path = "/home/pinagm/dev/Udacity_DataAnalyst_NanoDegree/D502 - Capstone Project/Kaggle Data/filtered_stock_data_quarterly.csv"
    quarterly_kag_df.to_csv(output_path, index=False)
    print(f"Quarterly data saved to {output_path}")


In [ ]:
def load_excel_if_exists(filepath: str) -> pd.DataFrame:
    """
    If the specified Excel file exists, read it into a DataFrame and print its head.
    Otherwise, print a 'File not found' message and return None.
    
    Parameters
    ----------
    filepath : str
        Path to the Excel file.
    
    Returns
    -------
    pd.DataFrame or None
        The loaded DataFrame, or None if the file does not exist.
    """
    if os.path.exists(filepath):
        df = pd.read_excel(filepath)
        print(f"Loaded Excel file: {filepath}")
        print(df.head())
        return df
    else:
        print(f"File not found: {filepath}")
        return None

def load_csv_if_exists(filepath: str) -> pd.DataFrame:
    """
    If the specified CSV file exists, read it into a DataFrame and print its head.
    Otherwise, print a 'File not found' message and return None.
    
    Parameters
    ----------
    filepath : str
        Path to the CSV file.
    
    Returns
    -------
    pd.DataFrame or None
        The loaded DataFrame, or None if the file does not exist.
    """
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        print(f"Loaded CSV file: {filepath}")
        print(df.head())
        return df
    else:
        print(f"File not found: {filepath}")
        return None


if __name__ == "__main__":
    current_date = datetime.now().strftime('%m-%d-%Y')
    bea_excel = f'BEA Data/bea-gdp-by-industry-filtered-{current_date}.xlsx'
    kag_csv = 'Kaggle Data/filtered_stock_data_quarterly.csv'
    
    bea_excel_df = load_excel_if_exists(bea_excel)
    kag_csv_df = load_csv_if_exists(kag_csv)


In [ ]:
roman_map = {'I': '1', 'II': '2', 'III': '3', 'IV': '4'}

def parse_quarter_str(qstr: str) -> pd.Timestamp:
    """
    Convert strings like '2020QI' or '2021QIII' into the end-of-quarter date:
    e.g. '2020-03-31'.

    Returns a pd.Timestamp, or None if format is invalid.
    """
    match = re.match(r"^(\d{4})Q(I|II|III|IV)$", qstr)
    if match:
        year_str = match.group(1)           # e.g. '2020'
        roman_quarter = match.group(2)      # e.g. 'II'
        year = int(year_str)
        quarter_num = int(roman_map[roman_quarter])  # 1..4
        period = pd.Period(year=year, quarter=quarter_num, freq='Q')
        return period.end_time
    else:
        return None

def melt_bea_quarters(
    df: pd.DataFrame,
    id_cols: list = ["Industry","IndustryDescription","Sector"]
) -> pd.DataFrame:
    """
    Melt a wide BEA DataFrame with quarter columns (e.g., '2020QI','2020QII',...)
    into a long format, creating a 'QuarterEnd' column for each quarter.
    """
    # Identify quarter columns that contain 'Q' but aren't in id_cols
    quarter_cols = [
        c for c in df.columns
        if c not in id_cols
        and 'Q' in c
    ]

    # Melt from wide to long
    melted = df.melt(
        id_vars=id_cols,
        value_vars=quarter_cols,
        var_name='QuarterStr',
        value_name='GDP'
    )

    # Convert QuarterStr to a real end-of-quarter date
    melted['QuarterEnd'] = melted['QuarterStr'].apply(parse_quarter_str)

    # Drop rows with invalid QuarterStr
    melted.dropna(subset=['QuarterEnd'], inplace=True)

    # Convert QuarterEnd to last day of that quarter
    melted['QuarterEnd'] = (
        melted['QuarterEnd'].dt.to_period('Q')
                             .dt.to_timestamp(freq='Q')
    )

    # Drop QuarterStr
    melted.drop(columns=['QuarterStr'], inplace=True)

    return melted

def convert_date_to_quarter_end(
    df: pd.DataFrame,
    date_col: str = "Date",
    out_col: str = "QuarterEnd"
) -> pd.DataFrame:
    """
    Convert a date column to the last day of its quarter in a new column (out_col).
    """
    df_copy = df.copy()
    df_copy[date_col] = pd.to_datetime(df_copy[date_col], errors='coerce')
    df_copy[out_col] = (
        df_copy[date_col].dt.to_period('Q')
                           .dt.to_timestamp(freq='Q')
    )
    return df_copy

def load_filtered_bea_file(folder: str = "BEA Data", keyword: str = "filtered") -> pd.DataFrame:
    """
    Search 'folder' for an .xlsx file containing 'keyword' in its filename.
    Return a DataFrame loaded from the first match, or None if none found.
    """
    pattern = os.path.join(folder, f"*{keyword}*.xlsx")
    matching_files = glob.glob(pattern)
    if not matching_files:
        print(f"No .xlsx file with '{keyword}' found in '{folder}'.")
        return None

    first_match = matching_files[0]
    print(f"Reading '{first_match}'...")
    return pd.read_excel(first_match)

def load_csv_if_exists(filepath: str) -> pd.DataFrame:
    """
    If the CSV file at 'filepath' exists, read it into a DataFrame and show its head.
    Otherwise, return None.
    """
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        print(f"Loaded CSV file: {filepath}")
        print(df.head())
        return df
    else:
        print(f"File not found: {filepath}")
        return None

if __name__ == "__main__":

    # 1) Load the "filtered" BEA Excel file automatically
    bea_excel_df = load_filtered_bea_file("BEA Data", keyword="filtered")
    if bea_excel_df is None:
        print("Cannot proceed: No filtered BEA file found.")
    else:
        print("Loaded BEA Excel data (head):")
        print(bea_excel_df.head())

        # 2) Melt the wide BEA DataFrame
        bea_melted = melt_bea_quarters(bea_excel_df)
        print("\nMelted BEA Data (head):")
        print(bea_melted.head(10))

    # 3) Load the Kaggle CSV if it exists
    kag_csv_path = "Kaggle Data/filtered_stock_data_quarterly.csv"
    kag_csv_df = load_csv_if_exists(kag_csv_path)
    if kag_csv_df is None:
        print("Cannot proceed: Kaggle CSV not found.")
    else:
        print("\nKaggle CSV initial (head):")
        print(kag_csv_df.head())

        # 4) Convert Kaggle date col to quarter-end
        kag_csv_df = convert_date_to_quarter_end(kag_csv_df, date_col="Date", out_col="QuarterEnd")
        print("\nKaggle CSV after quarter-end conversion (head):")
        print(kag_csv_df.head(10))


In [ ]:
def annotate_and_combine_ssga_data(
    ssga_df_list: List[pd.DataFrame],
    etf_list: List[str],
    etf_to_sector: Dict[str, str]
) -> pd.DataFrame:
    """
    Annotate each SSGA DataFrame with the corresponding ETF ticker and sector,
    then concatenate them all into one combined DataFrame.

    Parameters
    ----------
    ssga_df_list : List[pd.DataFrame]
        A list of DataFrames, one per ETF, typically from fetch_and_process_etf_data().
    etf_list : List[str]
        The list of ETF tickers in the same order as ssga_df_list (e.g., ['XLI','XLK','XLE','XLB']).
    etf_to_sector : Dict[str, str]
        A mapping from ETF symbol to sector name (e.g., {'XLI':'Industrials','XLK':'Technology'}).

    Returns
    -------
    pd.DataFrame
        A single DataFrame containing all rows from ssga_df_list, with added columns:
        - HoldingTicker (renamed from original 'Ticker')
        - ETF_Ticker
        - Sector
    """
    updated_ssga_dfs = []

    # Loop over each ETF name + SSGA DataFrame
    for etf_name, df in zip(etf_list, ssga_df_list):
        # Rename 'Ticker' -> 'HoldingTicker' to avoid confusion
        df = df.rename(columns={'Ticker': 'HoldingTicker'}).copy()

        # Add a column for the ETF ticker
        df['ETF_Ticker'] = etf_name

        # Use the mapping to find the sector
        if etf_name in etf_to_sector:
            df['Sector'] = etf_to_sector[etf_name]
        else:
            df['Sector'] = 'Unknown'  # or raise an error/warning

        updated_ssga_dfs.append(df)

    # Concatenate all the updated DataFrames
    combined_ssga_df = pd.concat(updated_ssga_dfs, ignore_index=True)
    return combined_ssga_df


if __name__ == "__main__":
    etf_list = ['XLI', 'XLK', 'XLE', 'XLB']
    etf_to_sector_map = {
        'XLI': 'Industrials',
        'XLK': 'Technology',
        'XLE': 'Energy',
        'XLB': 'Materials'
    }
    
    combined_ssga_df = annotate_and_combine_ssga_data(
        ssga_df_list,
        etf_list,
        etf_to_sector_map
    )

    print("Combined SSGA DataFrame (head):")
    print(combined_ssga_df.head(10))
    print("ETF_Ticker values:", combined_ssga_df['ETF_Ticker'].unique())
    print("Sector values:", combined_ssga_df['Sector'].unique())
    
xli_holdings = combined_ssga_df[combined_ssga_df['ETF_Ticker'] == 'XLI']
print(xli_holdings.head(10))

In [ ]:
# Merge Kaggle DF with combined SSGA DF
merged_kag_ssga = pd.merge(
    kag_csv_df,  # from Kaggle
    combined_ssga_df, 
    left_on='Ticker',     # from Kaggle
    right_on='HoldingTicker',  # from SSGA
    how='left'           # keep all Kaggle rows, even if no match
)

print("After merging Kaggle + SSGA, shape:", merged_kag_ssga.shape)
print(merged_kag_ssga.head())
bea_sector_agg = (
    bea_melted
    .groupby(["Sector","QuarterEnd"], as_index=False)
    .agg({"GDP": "sum"}) 
)

final_merged_df = pd.merge(
    merged_kag_ssga,
    bea_sector_agg,
    on=["Sector","QuarterEnd"],
    how="inner"
)

print("Final shape:", final_merged_df.shape)
print(final_merged_df.tail())

# Group by Sector and describe
grouped_stats = final_merged_df.groupby('Sector')[['Close','Volume','GDP']].describe()
print(grouped_stats)

In [ ]:
sns.boxplot(data=final_merged_df, x='Sector', y='Close')
plt.title("Distribution of Closing Prices by Sector")
plt.show()

In [ ]:
# Group by ETF and Ticker, and sum the weights
grouped_df = final_merged_df.groupby(['ETF', 'Ticker']).agg({'Weight': 'sum'}).reset_index()

# Sort tickers within each ETF by weight in descending order and get the top 5
top_tickers_df = grouped_df.sort_values(by=['ETF', 'Weight'], ascending=[True, False]).groupby('ETF').head(5)

# Plotting the top 5 tickers for each ETF
etfs = top_tickers_df['ETF'].unique()

for etf in etfs:
    top_tickers = top_tickers_df[top_tickers_df['ETF'] == etf]['Ticker']
    plt.figure(figsize=(10, 6))
    
    for ticker in top_tickers:
        df_ticker = final_merged_df[final_merged_df['Ticker'] == ticker].copy()
        plt.plot(df_ticker['QuarterEnd'], df_ticker['Close'], marker='o', label=ticker)
    
    plt.title(f"Top 5 Tickers by Weight for {etf}")
    plt.xlabel("QuarterEnd")
    plt.ylabel("Close Price")
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()

In [ ]:
sector_qtr = (final_merged_df
    .groupby(['Sector','QuarterEnd'], as_index=False)
    .agg({'Close':'mean','GDP':'mean'})  # mean GDP is the same as the single GDP if aggregated, but you can do it so yolo
)

# Then plot a line chart
for sector_name in sector_qtr['Sector'].unique():
    sub = sector_qtr[sector_qtr['Sector'] == sector_name]
    plt.plot(sub['QuarterEnd'], sub['Close'], label=f"{sector_name} Avg Close")

plt.title("Average Quarterly Close by Sector")
plt.xlabel("QuarterEnd")
plt.ylabel("Average Close")
plt.legend()
plt.xticks(rotation=45)
plt.show()


In [ ]:
sector_qtr = (final_merged_df
    .groupby(['Sector','QuarterEnd'], as_index=False)
    .agg({'GDP':'mean'})  # mean GDP is the same as the single GDP if aggregated, but you can do it so yolo
)

# Then plot a line chart
for sector_name in sector_qtr['Sector'].unique():
    sub = sector_qtr[sector_qtr['Sector'] == sector_name]
    plt.plot(sub['QuarterEnd'], sub['GDP'], label=f"{sector_name} Avg GDP")

plt.title("Average Quarterly GDP by Sector")
plt.xlabel("QuarterEnd")
plt.ylabel("Average GDP")
plt.legend()
plt.xticks(rotation=45)
plt.show()


In [ ]:
# 1) Group final_merged_df by Sector, QuarterEnd, and compute mean GDP & mean Close
df_agg = (final_merged_df
          .groupby(['Sector','QuarterEnd'], as_index=False)
          .agg({
              'GDP': 'mean',    
              'Close': 'mean'   
          }))

print(df_agg.head())


In [ ]:
sectors = df_agg['Sector'].unique()
fig, axes = plt.subplots(nrows=len(sectors), ncols=1, figsize=(8, 4*len(sectors)), sharex=True)

if len(sectors) == 1:
    axes = [axes]

for ax, sector_name in zip(axes, sectors):
    sub = df_agg[df_agg['Sector'] == sector_name].sort_values('QuarterEnd')

    # Left axis: GDP
    ax.plot(sub['QuarterEnd'], sub['GDP'], color='blue', marker='o', label='GDP')
    ax.set_ylabel("GDP", color='blue')
    ax.tick_params(axis='y', labelcolor='blue')

    # Create a second axis sharing the same x
    ax2 = ax.twinx()
    ax2.plot(sub['QuarterEnd'], sub['Close'], color='red', marker='o', label='Avg Price')
    ax2.set_ylabel("Stock Price", color='red')
    ax2.tick_params(axis='y', labelcolor='red')

    ax.set_title(f"{sector_name} - GDP vs. Stock Price (Dual Axis)")
    ax.set_xlabel("QuarterEnd")
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
corr_cols = ['Close','Volume','Dividends','Weight','GDP']
corr_matrix = final_merged_df[corr_cols].corr()
print(corr_matrix)

sns.heatmap(corr_matrix, annot=True, cmap='viridis')
plt.title("Correlation Heatmap")
plt.show()


In [ ]:
for sector_name in final_merged_df['Sector'].unique():
    sub = final_merged_df[final_merged_df['Sector'] == sector_name]
    corr_val = sub['Close'].corr(sub['GDP'])
    print(f"Correlation between Close and GDP in {sector_name}: {corr_val}")


In [ ]:
# 1) Sort our DataFrame so groupby shifts occur in the correct chronological order
final_merged_df = final_merged_df.sort_values(by=['Sector', 'QuarterEnd'])

# Ensure 'Pct_Return' column is created
final_merged_df['Pct_Return'] = final_merged_df.groupby('Ticker')['Close'].pct_change()

# 2) Compute quarter-over-quarter GDP growth per sector
#    pct_change() calculates (current - previous) / previous
final_merged_df['GDP_growth'] = final_merged_df.groupby('Sector')['GDP'].pct_change()

# 3) Create a lag of 1 quarter for GDP growth
final_merged_df['GDP_growth_lag1'] = final_merged_df.groupby('Sector')['GDP_growth'].shift(1)

# 4) For each sector, run OLS with Pct_Return as the dependent variable
#    and last quarter's GDP growth as the predictor
sectors = final_merged_df['Sector'].unique()
for sector_name in sectors:
    # Filter to that sector, drop rows with NaN in either Pct_Return or GDP_growth_lag1
    sub_df = final_merged_df[(final_merged_df['Sector'] == sector_name) & 
                      (~final_merged_df['Pct_Return'].isna()) & 
                      (~final_merged_df['GDP_growth_lag1'].isna())]
    
    # Check we have enough data
    if len(sub_df) > 2:
        model = smf.ols("Pct_Return ~ GDP_growth_lag1", data=sub_df).fit()
        print(f"\n=== Sector: {sector_name} ===")
        print(model.summary())
    else:
        print(f"\n=== Sector: {sector_name} ===")
        print("Not enough data points after dropping NaNs.")

In [ ]:
final_merged_df = final_merged_df.sort_values(by=['Ticker','QuarterEnd'])
final_merged_df['Pct_Return'] = final_merged_df.groupby('Ticker')['Close'].pct_change()
final_merged_df['GDP_scaled'] = final_merged_df['GDP'] / 1000_000  # e.g., in billions
model_df = final_merged_df.dropna(subset=['Pct_Return','GDP_growth_lag1'])
result = smf.ols("Pct_Return ~ GDP_growth_lag1", data=model_df).fit()
print(result.summary())

In [ ]:
# Run the regression for each sector with enough data
for sector_name in model_df['Sector'].unique():
    sub = model_df[model_df['Sector'] == sector_name]
    if len(sub) > 10:  # need enough data points
        reg_res = smf.ols("Pct_Return ~ GDP_growth_lag1", data=sub).fit()
        print(f"\n=== Sector: {sector_name} ===")
        print(reg_res.summary())


In [ ]:
# Filter the DataFrame for the focus sectors
sns.lmplot(data=final_merged_df, x='GDP', y='Close', hue='Sector', ci=None)
plt.title("Close Price vs. GDP by Sector")
plt.show()


In [ ]:
# Filter to the 'Materials' sector and remove extreme Pct_Return values
df_mat = final_merged_df[final_merged_df['Sector'] == 'Materials'].dropna(subset=['Pct_Return','GDP_growth_lag1'])
df_mat = df_mat[(df_mat['Pct_Return'] > -0.3) & (df_mat['Pct_Return'] < 0.3)]
df_mat['GDP_growth_lag1_pct'] = df_mat['GDP_growth_lag1'] * 100
sns.regplot(data=df_mat, x='GDP_growth_lag1_pct', y='Pct_Return')
plt.title("Materials: Next-Quarter Return vs. Previous Quarter GDP")
plt.show()

In [ ]:
# Scale GDP from e.g. millions to billions
final_merged_df['GDP_bln'] = final_merged_df['GDP'] / 1e3

# Re-run the regression but replace GDP_lag1 with scaled version
final_merged_df['GDP_bln_lag1'] = final_merged_df.groupby('Sector')['GDP_bln'].shift(1)

model_mat_bln = smf.ols("Pct_Return ~ GDP_bln_lag1", data=final_merged_df[final_merged_df['Sector'] == 'Materials']).fit()
print(model_mat_bln.summary())

In [ ]:
# Sort the DataFrame by Sector and QuarterEnd
final_merged_df = final_merged_df.sort_values(by=['Sector','QuarterEnd'])
final_merged_df['GDP_growth'] = final_merged_df.groupby('Sector')['GDP'].pct_change()
final_merged_df['GDP_growth_lag1'] = final_merged_df.groupby('Sector')['GDP_growth'].shift(1)

model_mat_growth = smf.ols("Pct_Return ~ GDP_growth_lag1", data=final_merged_df[final_merged_df['Sector'] == 'Materials']).fit()
print(model_mat_growth.summary())

In [ ]:
# 1) Extract the features used in our model
X = df_mat[['GDP_growth_lag1']] # independent variable

# 2) True target values
y = df_mat['Pct_Return'] # dependent variable

# 3) Predicted values from the model
y_pred = result.predict(X) # y_pred = b0 + b1*X

# 4) Plot predicted vs. actual
plt.scatter(y_pred, y, alpha=0.7)
plt.xlabel("Predicted (Pct_Return)")
plt.ylabel("Actual (Pct_Return)")
plt.title("Predicted vs. Actual Quarterly Returns (OLS)")

# Adding a diagonal line of perfect prediction
lims = [min(y.min(), y_pred.min()), max(y.max(), y_pred.max())]
plt.plot(lims, lims, 'r--')  # dashed red diagonal
plt.xlim(lims)
plt.ylim(lims)

plt.show()


In [ ]:
residuals = y - y_pred

plt.scatter(y_pred, residuals, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')  # zero-residual line
plt.xlabel("Predicted (Pct_Return)")
plt.ylabel("Residuals (Actual - Predicted)")
plt.title("Residual Plot (OLS)")

plt.show()
